In [14]:
import pandas as pd
import numpy as np
import uuid
from pp_ds_ml_base.config.connector import BigQueryConnectorConfig
from pp_ds_ml_base.connectors.bigquery import BigQueryConnector
from sxope_ml_hcc_prediction.app_config import app_config

In [15]:
gbq_connector = BigQueryConnector(
    BigQueryConnectorConfig(credentials_path=f"{app_config.project_root}/secrets/credentials.json")
)

In [16]:
query = '''
select
distinct
member_id
from pp-import-staging.spanner_sync_ver_10.members_eligibility
left join pp-ds-ml-staging.ml_inference_metadata.model_predictions
on members_eligibility.member_id = model_predictions.object_id
where model_predictions.object_id is null
qualify DATE_SUB(max(eligible_month_date) over(), interval 1 month) = eligible_month_date
'''
df_members = gbq_connector.bulk_data_download(query)

In [17]:
df_members['member_id_hex'] = df_members['member_id'].apply(lambda x: x.hex())

In [18]:
df_members

,member_id,member_id_hex
0,b'\xe4\\\xf4\xd8 \x08B\xe2\x87:\xd8\x1e\xb2J\x...,e45cf4d8200842e2873ad81eb24a19f9
1,b'f\x011\xfd\xe8!N\xe0\xb0M)V\xc1-H\xea',660131fde8214ee0b04d2956c12d48ea
2,b'\xa5|\xbb6\x86;@\x98\x86\x82\x11\r^#\x16R',a57cbb36863b40988682110d5e231652
3,b'-+%\xd9\r\xeaN\xbc\xa8K\xc0\xa9\xc1?-C',2d2b25d90dea4ebca84bc0a9c13f2d43
4,b'm\x87l\x92&\x99KO\xabi\xa4S\x10\xa0\x96\xac',6d876c9226994b4fab69a45310a096ac
...,...,...
112034,"b'\x11\xcb\x8d4)=F\xa2\x83\xf7\xcf""2\xe7\x04\xb8'",11cb8d34293d46a283f7cf2232e704b8
112035,b'\x17~\x07z\xf4\xfdI\xf4\xb3y\xc46\xa5\xa6\xb...,177e077af4fd49f4b379c436a5a6b9c7
112036,b'\xc8\x10>%\teG\xd4\x8dn\xe0t\x84\x17\x95\x9d',c8103e25096547d48d6ee0748417959d
112037,"b'\x85Z,/U\x87C\xfd\xb8x|{>\x0e\x8e\xc7'",855a2c2f558743fdb8787c7b3e0e8ec7


In [19]:
dfs = []
step = 5000
for i in range(int(np.ceil(df_members.shape[0] / step))):
    df_members.loc[step*i:step*(i+1), "member_id_hex"].to_csv(
        f"{app_config.project_root}/artifacts_sample/member_list_{i}.txt",
        header=False, index=False
    )